<a href="https://colab.research.google.com/github/widnu/Predictive_Policing_DL/blob/master/nz_dl_mod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# from google.colab import files
# uploaded = files.upload()

from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
import pandas as pd 
df=pd.read_csv('gdrive/My Drive/Colab Notebooks/dataset/nz_crime_dataset.csv', encoding='utf-8-sig')

In [13]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167347 entries, 0 to 1167346
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   DATE_NO_TIME               1167347 non-null  object 
 1   DATE_TIME                  1167347 non-null  object 
 2   DAY_AREA_CRIME_COUNT_ROLL  1167347 non-null  float64
 3   DAY_AREA_CRIME_COUNT       1167347 non-null  int64  
 4   MONTH_AREA_CRIME_COUNT     1167347 non-null  int64  
 5   YEAR_AREA_CRIME_COUNT      1167347 non-null  int64  
 6   TIME_SINCE_LAST_CRIME      1167347 non-null  float64
 7   MONTH                      1167347 non-null  int64  
 8   QUARTER                    1167347 non-null  float64
 9   DAY_OF_WEEK                1167347 non-null  object 
 10  DAY                        1167347 non-null  float64
 11  HOUR                       1167347 non-null  float64
 12  HOUR_PARTITION             1167347 non-null  float64
 13  MESHBLOCK   

,DAY_AREA_CRIME_COUNT_ROLL,DAY_AREA_CRIME_COUNT,MONTH_AREA_CRIME_COUNT,YEAR_AREA_CRIME_COUNT,TIME_SINCE_LAST_CRIME,MONTH,QUARTER,DAY,HOUR,HOUR_PARTITION,MESHBLOCK,WEAPON_TYPE,RISK
count,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06,1.167347e+06
mean,1.740339e+04,4.102593e+03,3.252062e+01,3.397887e+02,1.281016e-01,6.536036e+00,2.512512e+00,4.694526e+00,1.310084e+01,1.205440e+01,1.302863e+06,9.731468e-03,1.004760e+00
std,1.485673e+03,2.827224e+03,3.752400e+01,4.161401e+02,7.455829e+00,3.552330e+00,1.149184e+00,1.891168e+00,4.183061e+00,4.167649e+00,8.577749e+05,1.042445e-01,7.758172e-01
min,1.418400e+04,1.179000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-1.461000e+03,0.000000e+00,0.000000e+00
25%,1.655300e+04,1.628000e+03,1.000000e+01,9.300000e+01,0.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,1.262178e+01,1.200000e+01,6.075000e+05,0.000000e+00,0.000000e+00
50%,1.718400e+04,1.971000e+03,1.900000e+01,1.960000e+02,0.000000e+00,7.000000e+00,3.000000e+00,6.000000e+00,1.317855e+01,1.200000e+01,1.167000e+06,0.000000e+00,1.000000e+00
75%,1.789000e+04,7.184000e+03,3.900000e+01,3.970000e+02,0.000000e+00,1.000000e+01,4.000000e+00,6.000000e+00,1.400000e+01,1.200000e+01,2.029200e+06,0.000000e+00,2.000000e+00
max,2.263600e+04,8.902000e+03,2.760000e+02,2.294000e+03,5.770000e+02,1.200000e+01,4.000000e+00,7.000000e+00,2.300000e+01,2.100000e+01,3.210003e+06,2.000000e+00,2.000000e+00


In [14]:
#######################################################
# Count total NaN at each column in DataFrame
print(df.isnull().sum())

# Count all NaN in a DataFrame (both columns & Rows)
print(df.isnull().sum().sum())

# erase every row (axis=0) that has "any" Null value in it.
df = df.dropna(how='any',axis=0) 

DATE_NO_TIME                 0
DATE_TIME                    0
DAY_AREA_CRIME_COUNT_ROLL    0
DAY_AREA_CRIME_COUNT         0
MONTH_AREA_CRIME_COUNT       0
YEAR_AREA_CRIME_COUNT        0
TIME_SINCE_LAST_CRIME        0
MONTH                        0
QUARTER                      0
DAY_OF_WEEK                  0
DAY                          0
HOUR                         0
HOUR_PARTITION               0
MESHBLOCK                    0
AREA_0                       0
AREA_1                       0
WEAPON_TYPE                  0
CRIME_TYPE                   0
RISK                         0
dtype: int64
0


In [15]:
#######################################################
feature_var = ['DAY_AREA_CRIME_COUNT_ROLL', 'MONTH', 'DAY', 'QUARTER', 'HOUR_PARTITION', 'AREA_0', 'AREA_1', 'WEAPON_TYPE', 'CRIME_TYPE']
response_var = 'RISK'
#######################################################
X = df[feature_var]
y = df.pop(response_var)
# y = df[response_var]

In [16]:
########################################
# Encode categorical variables
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder()
oe.fit(X)
X = oe.transform(X)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=5)
# X = pca.fit_transform(X)

# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X = sc.fit_transform(X)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# from keras.utils import to_categorical
# y = to_categorical(y)

#######################################################
from imblearn.over_sampling import SMOTE
from collections import Counter
counter = Counter(y)
print(counter)
# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

#######################################################
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 5)

# Reserve 10,000 samples for validation
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

#######################################################
#df_X = pd.DataFrame(data=X)
#df_y = pd.DataFrame(data=y)

Counter({1: 464704, 2: 354100, 0: 348543})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Counter({0: 464704, 1: 464704, 2: 464704})


In [17]:
import tensorflow as tf
#dataset = tf.data.Dataset.from_tensor_slices((df_X.values, df_y.values))

In [18]:
#train_dataset = dataset.shuffle(len(df_X)).batch(1)

In [19]:
def get_compiled_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(3, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model


In [20]:
model = get_compiled_model()

history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=256,
    validation_data=(X_val, y_val),
)

history.history
model.summary()

# https://www.dlology.com/blog/how-to-choose-last-layer-activation-and-loss-function/

##################################################
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)

Epoch 1/256
15092/15092 [==============================] - 39s 3ms/step - loss: 0.9698 - accuracy: 0.5128 - val_loss: 0.9043 - val_accuracy: 0.5635
Epoch 2/256
15092/15092 [==============================] - 42s 3ms/step - loss: 0.8696 - accuracy: 0.5802 - val_loss: 0.8509 - val_accuracy: 0.5926
Epoch 3/256
15092/15092 [==============================] - 39s 3ms/step - loss: 0.8268 - accuracy: 0.6058 - val_loss: 0.8086 - val_accuracy: 0.6153
Epoch 4/256
15092/15092 [==============================] - 39s 3ms/step - loss: 0.7997 - accuracy: 0.6220 - val_loss: 0.7939 - val_accuracy: 0.6213
Epoch 5/256
15092/15092 [==============================] - 39s 3ms/step - loss: 0.7794 - accuracy: 0.6340 - val_loss: 0.7802 - val_accuracy: 0.6341
Epoch 6/256
15092/15092 [==============================] - 39s 3ms/step - loss: 0.7653 - accuracy: 0.6413 - val_loss: 0.7747 - val_accuracy: 0.6346
Epoch 7/256
15092/15092 [==============================] - 40s 3ms/step - loss: 0.7539 - accuracy: 0.6475 - val_